# Задание
Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.
Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.
Часть открытых данных есть, например, на сайте archive.org:


In [ ]:
import csv

with open('file.dat') as dat_file, open('file.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.
Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.
Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.
Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.
Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.
При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:


In [14]:
import pandas as pd
import numpy as np

In [15]:
df=pd.read_csv('file.csv')
df1 = df.sample(100000)
X=np.array((df1['latitude'],df1['longitude']))
X=X.T

In [6]:
from sklearn.cluster import MeanShift

In [16]:
cls=MeanShift(bandwidth=0.1)
cls.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [51]:
lab=cls.labels_
cnt=cls.cluster_centers_
unique, counts = np.unique(lab, return_counts=True)
lab_dict=dict(zip(unique, counts))
my_list=[]
for key, value in lab_dict.items():
    if (value<15):
        my_list.append(key)
    else: continue

In [52]:
new=np.delete(cnt,my_list,axis=0)

In [55]:
new_df = pd.DataFrame(new)
new_df.to_csv("file_path.csv") 

In [57]:
coor=np.array([[33.751277, -118.188740], [25.867736, -80.324116], [51.503016, -0.075479], [52.378894, 4.885084], [39.366487, 117.036146],[-33.868457, 151.205134]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.
Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.
Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.


In [65]:
from scipy.spatial import distance
mas_dist=distance.cdist(coor, new, 'euclidean')
mas_dist=mas_dist.T

In [79]:
min_dist=np.amin(mas_dist, axis = 1)

(624,)

In [82]:
my_list=[]
k=1000000000
for i in range(20):
    ind=min_dist.argmin()
    my_list.append(new[ind,:])
    min_dist[ind]=k
    

In [83]:
my_list

[array([-33.86347483, 151.20509035]),
 array([52.37123628,  4.89178318]),
 array([ 25.84975577, -80.30635146]),
 array([51.50371408, -0.1258998 ]),
 array([  33.8124583 , -118.14586751]),
 array([ 25.78731678, -80.21530739]),
 array([ 26.00582704, -80.21172905]),
 array([  33.90163134, -118.06400584]),
 array([ 26.13676891, -80.34598257]),
 array([  33.9209625 , -118.41349896]),
 array([ 26.10990246, -80.15800371]),
 array([  33.81083217, -117.89335083]),
 array([  34.05956509, -118.24768905]),
 array([  33.67634091, -117.85672067]),
 array([  34.07052493, -118.02341625]),
 array([  34.03328208, -118.43609127]),
 array([  34.1246087 , -118.12001983]),
 array([ 26.22695082, -80.19687934]),
 array([  34.05927774, -117.91820807]),
 array([  34.12968493, -118.35371205])]